In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import psrchive

import pipe

In [ ]:
g = list(sorted(glob(pipe.wsrt_obs_glob)))
s = 0
for d in g:
    try:
        M = pipe.process_wsrt_day(d)
        if M is None:
            print "skipping %s" % d
            continue
        print "success in %s:" % d, M["name"]
        s += 1
    except subprocess.CalledProcessError as e:
        print "psradd failure in %s:" % d, e
    except pipe.DiscontinuityError:
        traceback.print_exc()
print s, len(g)

success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Apr2012/J0337+1715-1380: 56018.55_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Apr2013/J0337+1715-1380: 56383.41_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Aug2012/J0337+1715-1380: 56140.36_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Dec2012/J0337+1715-1380: 56262.88_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Feb2013/J0337+1715-1380: 56324.62_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jan2013/J0337+1715-1380: 56293.87_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jul2012/J0337+1715-1380: 56109.30_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jul2013/J0337+1715-1380a: 56474.19_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jul2013/J0337+1715-1380b: 56474.37_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw

Traceback (most recent call last):
  File "<ipython-input-15-5e6b07c5d6a6>", line 5, in <module>
    M = pipe.process_wsrt_day(d)
  File "pipe.py", line 507, in process_wsrt_day
    raise pickle.load(f)
DiscontinuityError: Observation in directory /psr_archive/hessels/archibald/0337+17/raw/WSRT/03Oct2012/J0337+1715-1380 appears not to be contiguous: total integration time 3559.977216 but start-to-end time 3590.004430


success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Aug2012/J0337+1715-1380a: 56143.15_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Aug2012/J0337+1715-1380b: 56143.37_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Feb2012/J0337+1715-1380: 55961.86_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Feb2012/J0337+1715-385: 55961.87_WSRT_350
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Jan2013/J0337+1715-1380: 56296.82_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Jul2012/J0337+1715-1380: 56112.44_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Jul2013/J0337+1715-1380: 56477.41_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Jun2012/J0337+1715-1380: 56082.52_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/WSRT/04Mar2012/J0337+1715-1380: 55990.59_WSRT_1400
success in /psr_archive/hessels/archibald/0337+17/raw/W

In [12]:
!ls /psr_archive/hessels/archibald/0337+17/raw/WSRT/*/*/failure.pickle

/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Apr2012/J0337+1715-1380/failure.pickle


In [13]:
!rm /psr_archive/hessels/archibald/0337+17/raw/WSRT/*/*/failure.pickle

In [5]:
!ls /psr_archive/hessels/archibald/0337+17/raw/WSRT/*/*/success

/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Apr2012/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Apr2013/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Aug2012/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Dec2012/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Feb2013/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jan2013/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jul2012/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jul2013/J0337+1715-1380a/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jul2013/J0337+1715-1380b/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jun2012/J0337+1715-1380/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jun2013/J0337+1715-1380a/success
/psr_archive/hessels/archibald/0337+17/raw/WSRT/01Jun2013/J0337+1715-1380b/success
/psr_archive

In [6]:
!rm /psr_archive/hessels/archibald/0337+17/raw/WSRT/*/*/success